In [1]:
# Preprocessing — STEP 1: remove duplicates & drop identifier
import pandas as pd
pd.set_option('display.max_columns', None)

# load dataset (adjust path if needed)
df = pd.read_csv("../data/Faculty_Attrition_Synthetic_Dataset.csv")

print("Before cleaning:")
print("Shape:", df.shape)
print("Duplicate rows:", df.duplicated().sum())

# drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

# drop identifier column 
if "faculty_id" in df.columns:
    df = df.drop(columns=["faculty_id"])

print("\nAfter removing duplicates and dropping faculty_id:")
print("Shape:", df.shape)
print("Duplicate rows (now):", df.duplicated().sum())

# showing top 5 rows 
display(df.head())


Before cleaning:
Shape: (15200, 17)
Duplicate rows: 198

After removing duplicates and dropping faculty_id:
Shape: (15002, 16)
Duplicate rows (now): 0


,academic_rank,tenure_status,years_at_institution,base_salary,teaching_load,research_funding,institution_type,department_size,admin_support,work_life_balance,department_collaboration,promotion_opportunities,publications_last_3_years,student_evaluation_avg,job_market_alternatives,left_institution
0,Full Professor,Tenure-Track,8,83717.243906,5,23973.839447,Community College,Large,8,8.188558,2.827861,5.735050,7.0,3.429113,5.155787,0
1,Assistant Professor,Non-Tenure,3,45611.604269,9,21929.218175,Liberal Arts College,Small,3,2.369343,5.986704,5.598330,3.0,3.743535,1.000000,0
2,Associate Professor,Tenured,0,69397.334987,8,10317.051887,Technical Institute,Large,7,5.731299,2.289629,2.026023,4.0,4.849195,4.904489,0
3,Assistant Professor,Tenure-Track,7,57011.909828,5,50863.124935,Research University,Large,6,6.769015,9.967248,6.179585,3.0,4.328435,3.509004,1
4,Assistant Professor,Tenure-Track,15,57859.462561,7,47180.376820,Technical Institute,Medium,9,2.324824,6.939790,7.469714,4.0,4.031883,5.387402,1


In [4]:
# Preprocessing — STEP 2: missing-value analysis, indicators, and imputation
from sklearn.impute import SimpleImputer

# 1) Missing counts before
print("Missing counts BEFORE:")
missing_before = df.isna().sum().sort_values(ascending=False)
display(missing_before[missing_before > 0])

# 2) Create missing-indicator columns for any column that has missing values
cols_with_missing = missing_before[missing_before > 0].index.tolist()
for col in cols_with_missing:
    df[f"{col}_missing_flag"] = df[col].isna().astype(int)

print(f"\nCreated missing indicator flags for: {cols_with_missing}")

# 3) Separate numeric and categorical (exclude target)
target_col = "left_institution"
if target_col in df.columns:
    features = df.drop(columns=[target_col])
else:
    features = df.copy()

num_cols = features.select_dtypes(include=['number']).columns.tolist()
# returns columns with 'include' data type
cat_cols = features.select_dtypes(include=['object', 'category']).columns.tolist()

# remove any newly created flag cols from numeric/categorical lists if mis-typed
num_cols = [c for c in num_cols if not c.endswith("_missing_flag")]
cat_cols = [c for c in cat_cols if not c.endswith("_missing_flag")]

print("\nNumeric columns detected:", num_cols)
print("Categorical columns detected:", cat_cols)

# 4) Impute numeric with median, categorical with mode (in-place)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

if num_cols:
    df[num_cols] = num_imputer.fit_transform(df[num_cols])

if cat_cols:
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# 5) Verify no missing remain in feature columns (target should not be missing)
print("\nMissing counts AFTER imputation (features only):")
missing_after = df.drop(columns=[target_col]).isna().sum().sort_values(ascending=False)
display(missing_after[missing_after > 0])



Missing counts BEFORE:


Series([], dtype: int64)


Created missing indicator flags for: []

Numeric columns detected: ['years_at_institution', 'base_salary', 'teaching_load', 'research_funding', 'admin_support', 'work_life_balance', 'department_collaboration', 'promotion_opportunities', 'publications_last_3_years', 'student_evaluation_avg', 'job_market_alternatives']
Categorical columns detected: ['academic_rank', 'tenure_status', 'institution_type', 'department_size']

Missing counts AFTER imputation (features only):


Series([], dtype: int64)

In [5]:
# Preprocessing — STEP 3: outlier detection, flags, winsorization (1/99 clipping), and scaling
from sklearn.preprocessing import RobustScaler

target = 'left_institution'
num_cols = df.select_dtypes(include=['number']).columns.tolist()
# remove target and any flag cols from the numeric list if present
num_cols = [c for c in num_cols if c not in [target] and not c.endswith("_missing_flag")]

print("Numeric columns to process:", num_cols, "\n")

# 1) Show initial summary
print("Initial numeric summary (first 8 cols):")
display(df[num_cols].describe().T.iloc[:, :6])

# 2) IQR-based outlier detection counts
outlier_counts = {}
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    mask = (df[col] < lower) | (df[col] > upper)
    outlier_counts[col] = int(mask.sum())
    
outlier_counts_sorted = dict(sorted(outlier_counts.items(), key=lambda x: x[1], reverse=True))
print("Outlier counts (IQR rule) per numeric column:")
for k, v in outlier_counts_sorted.items():
    print(f"  {k}: {v}")

# 3) Create outlier flags for columns that have >0 outliers
outlier_flag_cols = []
for col, cnt in outlier_counts.items():
    if cnt > 0:
        flag_col = f"{col}_outlier_flag"
        df[flag_col] = ((df[col] < (df[col].quantile(0.25) - 1.5*(df[col].quantile(0.75)-df[col].quantile(0.25)))) |
                        (df[col] > (df[col].quantile(0.75) + 1.5*(df[col].quantile(0.75)-df[col].quantile(0.25))))).astype(int)
        outlier_flag_cols.append(flag_col)

print("\nCreated outlier flag columns for:", outlier_flag_cols)

# 4) Winsorize / cap values at 1st and 99th percentiles (adjustable)
lower_p = 0.01
upper_p = 0.99
caps = {}
for col in num_cols:
    low = df[col].quantile(lower_p)
    high = df[col].quantile(upper_p)
    caps[col] = (low, high)
    df[col] = df[col].clip(lower=low, upper=high)

print("\nApplied clipping at 1st and 99th percentiles for numeric columns.")

# 5) Scaling with RobustScaler
scaler = RobustScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

print("\nNumeric summary after clipping + Robust scaling (first 8 cols):")
display(df[num_cols].describe().T.iloc[:, :6])

# 6) verify outlier flags still meaningful
if outlier_flag_cols:
    print("\nSample rows where outlier flags are 1 (first 10):")
    display(df[df[outlier_flag_cols].any(axis=1)].head(10))
else:
    print("\nNo outlier flags created (no IQR outliers).")

# 7) Save list of numeric columns, flags, and scaler for later pipeline use
print("\nSaved lists for pipeline:")
print("  numeric_features:", num_cols)
print("  outlier_flag_columns:", outlier_flag_cols)


Numeric columns to process: ['years_at_institution', 'base_salary', 'teaching_load', 'research_funding', 'admin_support', 'work_life_balance', 'department_collaboration', 'promotion_opportunities', 'publications_last_3_years', 'student_evaluation_avg', 'job_market_alternatives'] 

Initial numeric summary (first 8 cols):


,count,mean,std,min,25%,50%
years_at_institution,15002.0,10.424810,7.292135,0.000000,5.000000,10.000000
base_salary,15002.0,61031.994343,15484.561439,37273.531332,51415.427707,57344.793838
teaching_load,15002.0,6.074390,2.045789,1.000000,5.000000,6.000000
research_funding,15002.0,32845.619862,24361.853906,16.095170,20563.447288,31013.062378
admin_support,15002.0,5.993401,1.976132,1.000000,5.000000,6.000000
work_life_balance,15002.0,5.506365,1.926064,1.000000,4.179213,5.505122
department_collaboration,15002.0,5.981169,1.947210,1.000000,4.655716,5.990321
promotion_opportunities,15002.0,5.003399,1.949363,1.000000,3.640900,4.982620
publications_last_3_years,15002.0,3.622050,2.109037,0.000000,2.000000,3.000000
student_evaluation_avg,15002.0,3.988674,0.487100,1.952134,3.663822,3.991181


Outlier counts (IQR rule) per numeric column:
  teaching_load: 389
  research_funding: 251
  base_salary: 242
  admin_support: 173
  publications_last_3_years: 126
  years_at_institution: 82
  student_evaluation_avg: 51
  work_life_balance: 0
  department_collaboration: 0
  promotion_opportunities: 0
  job_market_alternatives: 0

Created outlier flag columns for: ['years_at_institution_outlier_flag', 'base_salary_outlier_flag', 'teaching_load_outlier_flag', 'research_funding_outlier_flag', 'admin_support_outlier_flag', 'publications_last_3_years_outlier_flag', 'student_evaluation_avg_outlier_flag']

Applied clipping at 1st and 99th percentiles for numeric columns.

Numeric summary after clipping + Robust scaling (first 8 cols):


,count,mean,std,min,25%,50%
years_at_institution,15002.0,0.039935,0.721822,-1.000000,-0.500000,0.000000e+00
base_salary,15002.0,0.203374,0.748109,-0.890086,-0.358007,0.000000e+00
teaching_load,15002.0,0.035029,1.017130,-2.500000,-0.500000,0.000000e+00
research_funding,15002.0,0.059123,0.791514,-1.331606,-0.461413,0.000000e+00
admin_support,15002.0,-0.003300,0.988066,-2.500000,-0.500000,0.000000e+00
work_life_balance,15002.0,0.000471,0.729885,-1.707224,-0.502455,0.000000e+00
department_collaboration,15002.0,-0.002411,0.726493,-1.741900,-0.499651,0.000000e+00
promotion_opportunities,15002.0,0.006497,0.718347,-1.473568,-0.496436,-1.643108e-16
publications_last_3_years,15002.0,0.202862,0.689129,-1.000000,-0.333333,0.000000e+00
student_evaluation_avg,15002.0,-0.001418,0.729213,-1.732407,-0.494331,0.000000e+00



Sample rows where outlier flags are 1 (first 10):


,academic_rank,tenure_status,years_at_institution,base_salary,teaching_load,research_funding,institution_type,department_size,admin_support,work_life_balance,department_collaboration,promotion_opportunities,publications_last_3_years,student_evaluation_avg,job_market_alternatives,left_institution,research_funding_missing_flag,base_salary_missing_flag,work_life_balance_missing_flag,student_evaluation_avg_missing_flag,publications_last_3_years_missing_flag,years_at_institution_outlier_flag,base_salary_outlier_flag,teaching_load_outlier_flag,research_funding_outlier_flag,admin_support_outlier_flag,publications_last_3_years_outlier_flag,student_evaluation_avg_outlier_flag
11,Full Professor,Tenured,1.9,2.051111,0.0,1.127421,Research University,Large,-1.0,0.737494,0.701510,0.727369,1.333333,-0.349340,1.168426,0,0,0,0,0,0,1,0,0,0,0,0,0
13,Assistant Professor,Non-Tenure,0.2,-0.403341,-0.5,2.049259,Research University,Medium,1.0,-0.804410,-0.365341,0.439138,-0.333333,-0.911319,-0.574307,1,0,0,0,0,0,0,0,0,1,0,0,0
18,Assistant Professor,Tenure-Track,-1.0,-0.472880,-1.5,-0.249774,Technical Institute,Medium,-2.5,0.020718,0.962496,0.214052,0.666667,1.185929,0.598105,1,0,0,0,0,0,0,0,0,0,1,0,0
25,Full Professor,Tenure-Track,-0.1,2.263782,0.5,1.393813,Research University,Medium,-0.5,-0.575267,-0.236408,-0.869257,0.666667,-0.379819,-0.298363,0,0,0,0,0,0,0,1,0,0,0,0,0
55,Full Professor,Non-Tenure,0.1,2.298579,0.0,0.165014,Liberal Arts College,Large,1.0,-0.226799,-0.041802,-1.206420,1.000000,-0.009958,0.301893,0,0,0,0,0,0,0,1,0,0,0,0,0
65,Full Professor,Tenure-Track,0.9,1.888247,-1.5,0.000000,Liberal Arts College,Medium,0.0,1.119363,0.332313,0.849212,2.000000,0.588038,-1.198746,0,1,0,0,0,0,0,0,0,0,0,1,0
80,Full Professor,Tenured,1.9,2.298579,-1.0,-0.675510,Liberal Arts College,Small,0.5,-0.647025,-0.269778,1.365244,1.333333,-0.385476,-0.881096,0,0,0,0,0,0,0,1,0,0,0,0,0
86,Assistant Professor,Tenured,1.9,0.501479,-1.0,-0.185808,Research University,Large,-1.0,0.000000,-0.544444,0.755210,-0.333333,0.383756,-0.476619,0,0,0,1,0,0,1,0,0,0,0,0,0
87,Full Professor,Tenured,0.3,1.231396,-1.0,2.271486,Research University,Large,0.5,-0.725624,0.490516,0.242155,0.333333,0.301360,-0.846159,1,0,0,0,0,0,0,0,0,1,0,0,0
91,Assistant Professor,Tenure-Track,1.9,-0.044098,2.0,0.524606,Technical Institute,Small,1.0,-0.783338,-0.090173,-0.318066,1.333333,-0.000181,-0.382627,0,0,0,0,0,0,1,0,0,0,0,0,0



Saved lists for pipeline:
  numeric_features: ['years_at_institution', 'base_salary', 'teaching_load', 'research_funding', 'admin_support', 'work_life_balance', 'department_collaboration', 'promotion_opportunities', 'publications_last_3_years', 'student_evaluation_avg', 'job_market_alternatives']
  outlier_flag_columns: ['years_at_institution_outlier_flag', 'base_salary_outlier_flag', 'teaching_load_outlier_flag', 'research_funding_outlier_flag', 'admin_support_outlier_flag', 'publications_last_3_years_outlier_flag', 'student_evaluation_avg_outlier_flag']


In [6]:
# Preprocessing — STEP 4: Identify and prepare categorical columns for OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

target = "left_institution"

# Identify categorical columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("Categorical columns detected:")
print(categorical_cols, "\n")

# check the unique values for each categorical column
for col in categorical_cols:
    print(f"Column: {col}")
    print(" Unique values:", df[col].unique())
    print("-" * 50)

# These columns will be OneHotEncoded inside ColumnTransformer (next step)
print("\nThese columns will be OneHotEncoded as part of the pipeline.")

# Let's also gather numeric columns again (excluding target, missing flags, outlier flags)
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
numeric_cols = [c for c in numeric_cols if not c.endswith("_missing_flag") 
                                         and not c.endswith("_outlier_flag")
                                         and c != target]

missing_flag_cols = [c for c in df.columns if c.endswith("_missing_flag")]
outlier_flag_cols = [c for c in df.columns if c.endswith("_outlier_flag")]

print("\nFinal feature groups:")
print(" Numeric columns:", numeric_cols)
print(" Missing-flag columns:", missing_flag_cols)
print(" Outlier-flag columns:", outlier_flag_cols)
print(" Categorical columns:", categorical_cols)


Categorical columns detected:
['academic_rank', 'tenure_status', 'institution_type', 'department_size'] 

Column: academic_rank
 Unique values: ['Full Professor' 'Assistant Professor' 'Associate Professor' 'Lecturer']
--------------------------------------------------
Column: tenure_status
 Unique values: ['Tenure-Track' 'Non-Tenure' 'Tenured']
--------------------------------------------------
Column: institution_type
 Unique values: ['Community College' 'Liberal Arts College' 'Technical Institute'
 'Research University']
--------------------------------------------------
Column: department_size
 Unique values: ['Large' 'Small' 'Medium']
--------------------------------------------------

These columns will be OneHotEncoded as part of the pipeline.

Final feature groups:
 Numeric columns: ['years_at_institution', 'base_salary', 'teaching_load', 'research_funding', 'admin_support', 'work_life_balance', 'department_collaboration', 'promotion_opportunities', 'publications_last_3_years', 

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

numeric_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols),
    ('flags', 'passthrough', missing_flag_cols + outlier_flag_cols)
])


In [ ]:
# STEP 5 — Train/Test Split + Build Preprocessor
from sklearn.model_selection import train_test_split

target = "left_institution"

X = df.drop(columns=[target])
y = df[target]

print("X shape:", X.shape)
print("y shape:", y.shape)

# Train-test split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("\nTraining set:", X_train.shape)
print("Test set:", X_test.shape)
print("Target distribution (train):")
print(y_train.value_counts(normalize=True))
print("Target distribution (test):")
print(y_test.value_counts(normalize=True))


categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_cols = X_train.select_dtypes(include=["number"]).columns.tolist()

# Remove flags from numeric
missing_flag_cols = [c for c in X_train.columns if c.endswith("_missing_flag")]
outlier_flag_cols = [c for c in X_train.columns if c.endswith("_outlier_flag")]

numeric_cols = [c for c in numeric_cols 
                if not c.endswith("_missing_flag") 
                and not c.endswith("_outlier_flag")]

print("\nCategorical columns:", categorical_cols)
print("Numeric columns:", numeric_cols)
print("Missing-flag columns:", missing_flag_cols)
print("Outlier-flag columns:", outlier_flag_cols)


numeric_pipeline = Pipeline([
    ("scaler", RobustScaler())
])

categorical_pipeline = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_cols),
    ("cat", categorical_pipeline, categorical_cols),
    ("flags", "passthrough", missing_flag_cols + outlier_flag_cols)
])

preprocessor.fit(X_train)

print("\nPreprocessor fitted on training data.")
